In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import requests as req
from fpdf import FPDF

In [34]:
import pandas as pd

#GET DATA:
PATH="./Input/country_population.csv"
def get_data(path=PATH):
    df=pd.read_csv(path,",")
    return df

In [142]:
import pandas as pd
import numpy as np


#CLEAN DATA:
def clean_data(df):
    col_drop=["Indicator Name","Indicator Code"]
    count_drop=[
        "Arab World",
        "Central Europe and the Baltics",
        "Caribbean small states",
        "East Asia & Pacific (excluding high income)",
        "Early-demographic dividend",
        'East Asia & Pacific',
        'Europe & Central Asia (excluding high income)',
        'Europe & Central Asia',
        'Euro area',
        'European Union',
        'Fragile and conflict affected situations',
        'High income',
        'Heavily indebted poor countries (HIPC)',
        'IBRD only',
        'IDA & IBRD total',
        'IDA total',
        'IDA blend',
        'IDA only',
        'Not classified',
        'Latin America & Caribbean (excluding high income)',
        'Latin America & Caribbean',
        'Least developed countries: UN classification',
        'Low income',
        'Lower middle income',
        'Low & middle income',
        'Late-demographic dividend',
        'Middle East & North Africa',
        'Middle income',
        'Middle East & North Africa (excluding high income)',
        'North America',
        'OECD members',
        'Other small states',
        'Pre-demographic dividend',
        'West Bank and Gaza',
        'Pacific island small states',
        'Post-demographic dividend',
        'South Asia',
        'Sub-Saharan Africa (excluding high income)',
        'Sub-Saharan Africa',
        'Small states',
        'East Asia & Pacific (IDA & IBRD countries)',
        'Europe & Central Asia (IDA & IBRD countries)',
        'Latin America & the Caribbean (IDA & IBRD countries)',
        'Middle East & North Africa (IDA & IBRD countries)',
        'South Asia (IDA & IBRD)',
        'Sub-Saharan Africa (IDA & IBRD countries)',
        'Upper middle income','World']
    cdf=df.copy()
    cdf=cdf.drop(col_drop, axis=1)
    bad_countries=cdf[cdf["Country Name"].isin(count_drop)==True].index
    cdf=cdf.drop(bad_countries,axis=0)
    cdf["Country Name"] = cdf["Country Name"].apply(lambda x: " ".join(x.split(",")[::-1]) if x in ["Congo, Dem. Rep.","Congo, Rep.","Korea, Rep.","Korea, Dem. People’s Rep."] else x)
    cdf["Country Name"] = cdf["Country Name"].apply(lambda x: x.split(",")[0])
    cdf=cdf.fillna(0)
    cdf["Country Name"] = cdf["Country Name"].apply(lambda x: x.upper().strip())
    return cdf

In [17]:
import pandas as pd
import numpy as np

#FILTER AND FIX DATA:
def filter_data(df,country="SPAIN",desde=2006,hasta=2016):
    lista_años_st=list(map(str,list(range(desde,hasta+1))))
    lista_total=["Country Name","Country Code"]+lista_años_st
    fdf=df[df["Country Name"]==(country.upper())][lista_total]
    return fdf

def fix_data(df):
    melt=pd.melt(df, id_vars=["Country Name","Country Code"], value_vars=df.columns[2:],var_name="Year",value_name="Population")
    melt=melt.sort_values(by=["Country Name", "Year"])
    return melt

    

In [18]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

#ANALYSE DATA:

def get_growth(df):
    gdf=df.copy()
    lista=[0]
    for i in range(1,len(df["Population"])):
        lista.append(((df["Population"][i]/df["Population"][i-1])-1)*100)
    gdf["Growth rate %"]=np.array(lista)
    return gdf[1:]

def plot_population(df):
    plt.figure(figsize=(18,6))
    plt.plot(df.groupby(["Year"]).sum()["Population"]/1000000, c='blue')
    plt.legend(loc=1)
    plt.title('Population by year')
    plt.xlabel('Year')
    plt.ylabel('Population (Millions)')
    plt.savefig('./Output/population.png')
    plt.show()
    return './Output/population.png'

def plot_growth(df):
    plt.figure(figsize=(18,6))
    plt.plot(df.groupby(["Year"]).sum()["Growth rate %"], c='blue')
    plt.legend(loc=1)
    plt.title('Population growth rate by year')
    plt.xlabel('Year')
    plt.ylabel('Growth rate %')
    plt.savefig('./Output/growth.png')
    plt.show()
    return './Output/growth.png'


In [68]:
import pandas as pd
import numpy as np
import requests as req

#API ENRICH:
BASE_URL="https://restcountries.eu/rest/v2/alpha/"
def get_country(df):
    name=df["Country Name"].unique()[0]
    return str(name)

def get_ccode(df):
    code=df["Country Code"].unique()[0]
    return str(code)

def get_capital(url,code):
    query_params={"fields":"capital"}
    res = req.get("{}{}".format(url,code),params=query_params)
    content=res.json()
    capital=content["capital"]
    return capital

def get_alt_names(url,code):
    query_params={"fields":"altSpellings"}
    res = req.get("{}{}".format(url,code),params=query_params)
    content=res.json()
    names=", ".join(content["altSpellings"])
    return names

def get_region(url,code):
    query_params={"fields":"region"}
    res = req.get("{}{}".format(url,code),params=query_params)
    content=res.json()
    region=content["region"]
    return region

def get_population(url,code):
    query_params={"fields":"population"}
    res = req.get("{}{}".format(url,code),params=query_params)
    content=res.json()
    pop=str(content["population"])
    return pop

def get_lang(url,code):
    query_params={"fields":"languages"}
    res = req.get("{}{}".format(url,code),params=query_params)
    content=res.json()
    lang_list=[l["name"] for l in content["languages"]]
    languages=", ".join(lang_list)
    return languages

def get_flag(url,code):
    query_params={"fields":"flag"}
    res = req.get("{}{}".format(url,code),params=query_params)
    content=res.json()
    img_url=content["flag"]

    pic_res = req.get(img_url)
    if pic_res.status_code == 200:
        with open("./Output/flag.svg", 'wb') as f:
            f.write(pic_res.content)
    return "./Output/flag.svg"

In [143]:
from fpdf import FPDF

#PDF GEN:

def make_PDF(cname,calt_names,ccapital,cregion,clanguages,ccurrpop,cpoplot,cgrplot):
    pdf=FPDF()
    pdf.add_page() 
    pdf.set_xy(0, 0)
    pdf.set_font('arial', 'B', 12)
    pdf.cell(60)
    pdf.cell(90, 10, cname, 0, 2, 'C')
    pdf.cell(90, 10, "Population and growth", 0, 2, 'C')
    pdf.cell(90, 10, " ", 0, 2, 'C')
    pdf.set_font('arial',"B" ,10)
    pdf.cell(-50)
    pdf.cell(90, 10, f"Alternative names: {calt_names}", 0, 2)
    pdf.cell(1)
    pdf.cell(0, 10, f"Capital: {ccapital}", 0, 2 )
    pdf.cell(-1)
    pdf.cell(0, 10, f"Region: {cregion}", 0, 2 )
    pdf.cell(1)
    pdf.cell(0, 10, f"Languages: {clanguages}", 0, 2 )
    pdf.cell(-1)
    pdf.cell(0, 10, f"Current population: {ccurrpop} citizens", 0, 2 )
    pdf.cell(90, 10, " ", 0, 2, 'C')
    pdf.cell(90, 10, " ", 0, 2, 'C')
    pdf.cell(50)
    pdf.cell(90, 10, "PLOTS", 0, 2, 'C')
    pdf.image(cpoplot, x=8, y=110, w=200, h=60, type = '', link = '')
    pdf.image(cgrplot, x=8, y=170, w=200, h=60, type = '', link = '')
    pdf.output(f'./Output/{cname}_report.pdf', 'F')

        


In [144]:
data=get_data()


In [145]:
clean=clean_data(data)

In [151]:
listalist(clean["Country Name"].unique())

['ARUBA',
 'AFGHANISTAN',
 'ANGOLA',
 'ALBANIA',
 'ANDORRA',
 'UNITED ARAB EMIRATES',
 'ARGENTINA',
 'ARMENIA',
 'AMERICAN SAMOA',
 'ANTIGUA AND BARBUDA',
 'AUSTRALIA',
 'AUSTRIA',
 'AZERBAIJAN',
 'BURUNDI',
 'BELGIUM',
 'BENIN',
 'BURKINA FASO',
 'BANGLADESH',
 'BULGARIA',
 'BAHRAIN',
 'BAHAMAS',
 'BOSNIA AND HERZEGOVINA',
 'BELARUS',
 'BELIZE',
 'BERMUDA',
 'BOLIVIA',
 'BRAZIL',
 'BARBADOS',
 'BRUNEI DARUSSALAM',
 'BHUTAN',
 'BOTSWANA',
 'CENTRAL AFRICAN REPUBLIC',
 'CANADA',
 'SWITZERLAND',
 'CHANNEL ISLANDS',
 'CHILE',
 'CHINA',
 "COTE D'IVOIRE",
 'CAMEROON',
 'DEM. REP. CONGO',
 'REP. CONGO',
 'COLOMBIA',
 'COMOROS',
 'CABO VERDE',
 'COSTA RICA',
 'CUBA',
 'CURACAO',
 'CAYMAN ISLANDS',
 'CYPRUS',
 'CZECH REPUBLIC',
 'GERMANY',
 'DJIBOUTI',
 'DOMINICA',
 'DENMARK',
 'DOMINICAN REPUBLIC',
 'ALGERIA',
 'ECUADOR',
 'EGYPT',
 'ERITREA',
 'SPAIN',
 'ESTONIA',
 'ETHIOPIA',
 'FINLAND',
 'FIJI',
 'FRANCE',
 'FAROE ISLANDS',
 'MICRONESIA',
 'GABON',
 'UNITED KINGDOM',
 'GEORGIA',
 'GHANA',


In [152]:
COUNTRIES=['ARUBA',
 'AFGHANISTAN',
 'ANGOLA',
 'ALBANIA',
 'ANDORRA',
 'UNITED ARAB EMIRATES',
 'ARGENTINA',
 'ARMENIA',
 'AMERICAN SAMOA',
 'ANTIGUA AND BARBUDA',
 'AUSTRALIA',
 'AUSTRIA',
 'AZERBAIJAN',
 'BURUNDI',
 'BELGIUM',
 'BENIN',
 'BURKINA FASO',
 'BANGLADESH',
 'BULGARIA',
 'BAHRAIN',
 'BAHAMAS',
 'BOSNIA AND HERZEGOVINA',
 'BELARUS',
 'BELIZE',
 'BERMUDA',
 'BOLIVIA',
 'BRAZIL',
 'BARBADOS',
 'BRUNEI DARUSSALAM',
 'BHUTAN',
 'BOTSWANA',
 'CENTRAL AFRICAN REPUBLIC',
 'CANADA',
 'SWITZERLAND',
 'CHANNEL ISLANDS',
 'CHILE',
 'CHINA',
 "COTE D'IVOIRE",
 'CAMEROON',
 'DEM. REP. CONGO',
 'REP. CONGO',
 'COLOMBIA',
 'COMOROS',
 'CABO VERDE',
 'COSTA RICA',
 'CUBA',
 'CURACAO',
 'CAYMAN ISLANDS',
 'CYPRUS',
 'CZECH REPUBLIC',
 'GERMANY',
 'DJIBOUTI',
 'DOMINICA',
 'DENMARK',
 'DOMINICAN REPUBLIC',
 'ALGERIA',
 'ECUADOR',
 'EGYPT',
 'ERITREA',
 'SPAIN',
 'ESTONIA',
 'ETHIOPIA',
 'FINLAND',
 'FIJI',
 'FRANCE',
 'FAROE ISLANDS',
 'MICRONESIA',
 'GABON',
 'UNITED KINGDOM',
 'GEORGIA',
 'GHANA',
 'GIBRALTAR',
 'GUINEA',
 'GAMBIA',
 'GUINEA-BISSAU',
 'EQUATORIAL GUINEA',
 'GREECE',
 'GRENADA',
 'GREENLAND',
 'GUATEMALA',
 'GUAM',
 'GUYANA',
 'HONG KONG SAR',
 'HONDURAS',
 'CROATIA',
 'HAITI',
 'HUNGARY',
 'INDONESIA',
 'ISLE OF MAN',
 'INDIA',
 'IRELAND',
 'IRAN',
 'IRAQ',
 'ICELAND',
 'ISRAEL',
 'ITALY',
 'JAMAICA',
 'JORDAN',
 'JAPAN',
 'KAZAKHSTAN',
 'KENYA',
 'KYRGYZ REPUBLIC',
 'CAMBODIA',
 'KIRIBATI',
 'ST. KITTS AND NEVIS',
 'REP. KOREA',
 'KUWAIT',
 'LAO PDR',
 'LEBANON',
 'LIBERIA',
 'LIBYA',
 'ST. LUCIA',
 'LIECHTENSTEIN',
 'SRI LANKA',
 'LESOTHO',
 'LITHUANIA',
 'LUXEMBOURG',
 'LATVIA',
 'MACAO SAR',
 'ST. MARTIN (FRENCH PART)',
 'MOROCCO',
 'MONACO',
 'MOLDOVA',
 'MADAGASCAR',
 'MALDIVES',
 'MEXICO',
 'MARSHALL ISLANDS',
 'MACEDONIA',
 'MALI',
 'MALTA',
 'MYANMAR',
 'MONTENEGRO',
 'MONGOLIA',
 'NORTHERN MARIANA ISLANDS',
 'MOZAMBIQUE',
 'MAURITANIA',
 'MAURITIUS',
 'MALAWI',
 'MALAYSIA',
 'NAMIBIA',
 'NEW CALEDONIA',
 'NIGER',
 'NIGERIA',
 'NICARAGUA',
 'NETHERLANDS',
 'NORWAY',
 'NEPAL',
 'NAURU',
 'NEW ZEALAND',
 'OMAN',
 'PAKISTAN',
 'PANAMA',
 'PERU',
 'PHILIPPINES',
 'PALAU',
 'PAPUA NEW GUINEA',
 'POLAND',
 'PUERTO RICO',
 'DEM. PEOPLE’S REP. KOREA',
 'PORTUGAL',
 'PARAGUAY',
 'FRENCH POLYNESIA',
 'QATAR',
 'ROMANIA',
 'RUSSIAN FEDERATION',
 'RWANDA',
 'SAUDI ARABIA',
 'SUDAN',
 'SENEGAL',
 'SINGAPORE',
 'SOLOMON ISLANDS',
 'SIERRA LEONE',
 'EL SALVADOR',
 'SAN MARINO',
 'SOMALIA',
 'SERBIA',
 'SOUTH SUDAN',
 'SAO TOME AND PRINCIPE',
 'SURINAME',
 'SLOVAK REPUBLIC',
 'SLOVENIA',
 'SWEDEN',
 'SWAZILAND',
 'SINT MAARTEN (DUTCH PART)',
 'SEYCHELLES',
 'SYRIAN ARAB REPUBLIC',
 'TURKS AND CAICOS ISLANDS',
 'CHAD',
 'TOGO',
 'THAILAND',
 'TAJIKISTAN',
 'TURKMENISTAN',
 'TIMOR-LESTE',
 'TONGA',
 'TRINIDAD AND TOBAGO',
 'TUNISIA',
 'TURKEY',
 'TUVALU',
 'TANZANIA',
 'UGANDA',
 'UKRAINE',
 'URUGUAY',
 'UNITED STATES',
 'UZBEKISTAN',
 'ST. VINCENT AND THE GRENADINES',
 'VENEZUELA',
 'BRITISH VIRGIN ISLANDS',
 'VIRGIN ISLANDS (U.S.)',
 'VIETNAM',
 'VANUATU',
 'SAMOA',
 'KOSOVO',
 'YEMEN',
 'SOUTH AFRICA',
 'ZAMBIA',
 'ZIMBABWE']